## Using GPT4All to explain SQL queries

In [2]:
import evadb
cursor = evadb.connect().cursor()

In [3]:
from gpt4all import GPT4All
model = GPT4All("orca-mini-3b.ggmlv3.q4_0.bin")

/Users/krishnathan/Documents/College/Fall 23/CS 6422/Project/evadb-project/evadb_env/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Found model file at  /Users/krishnathan/.cache/gpt4all/orca-mini-3b.ggmlv3.q4_0.bin


objc[21466]: Class GGMLMetalClass is implemented in both /Users/krishnathan/Documents/College/Fall 23/CS 6422/Project/evadb-project/evadb_env/lib/python3.9/site-packages/gpt4all/llmodel_DO_NOT_MODIFY/build/libreplit-mainline-metal.dylib (0x13dd14228) and /Users/krishnathan/Documents/College/Fall 23/CS 6422/Project/evadb-project/evadb_env/lib/python3.9/site-packages/gpt4all/llmodel_DO_NOT_MODIFY/build/libllamamodel-mainline-metal.dylib (0x13dbdc228). One of the two will be used. Which one is undefined.
llama.cpp: using Metal
llama.cpp: loading model from /Users/krishnathan/.cache/gpt4all/orca-mini-3b.ggmlv3.q4_0.bin
llama_model_load_internal: format     = ggjt v3 (latest)
llama_model_load_internal: n_vocab    = 32000
llama_model_load_internal: n_ctx      = 2048
llama_model_load_internal: n_embd     = 3200
llama_model_load_internal: n_mult     = 240
llama_model_load_internal: n_head     = 32
llama_model_load_internal: n_head_kv  = 32
llama_model_load_internal: n_layer    = 26
llama_model

### Explain what SELECT does

In [4]:
tokens = model.generate("Explain what a SELECT statement does", max_tokens=30, streaming=True)
print("".join(tokens))

 and how it works.
A SELECT statement is used to retrieve data from one or more tables in a database. It consists of several parts, including


In [5]:
tokens = list(model.generate("Explain what 'SELECT * FROM students' does", max_tokens=50, streaming=True))
print("".join(tokens))

.
The SELECT statement is used to retrieve data from one or more tables in a database. It requires the name of the table(s) you want to select data from and specifies which columns to include in the result set. The query can then


### Wrapper function to explain general query

In [6]:
def explain_sql(query_string: str, response_length: int) -> str:
    tokens = list(model.generate("Explain what the SQL query: '{}' does".format(query_string), max_tokens=response_length, streaming=True))
    text = "".join(tokens)
    return text

In [7]:
print(explain_sql("SELECT * FROM students", 50))

.
The SQL query 'SELECT * FROM students' is used to retrieve all columns (i.e., all data) from a table named "students". The asterisk (*) symbol means that we want to select all columns in the


Checking that this also works for INSERT and GROUP queries

In [12]:
print(explain_sql("INSERT INTO students (id, name) VALUES ('1', 'Krish')", 50))

.

The given SQL query is used to insert a new row into the table "students" with an ID of 1 and a name of "Krish". The "VALUES" keyword specifies which column(s) to insert data


In [13]:
print(
explain_sql(""" 
SELECT COUNT(CustomerID), Country
FROM Customers
GROUP BY Country
ORDER BY COUNT(CustomerID) DESC;""", 
50))

.


This particular model, orca-mini, struggles sometimes with explaining the INSERT and GROUP queries. For example, sometimes it will just return the string ".". I believe this is because the model is relatively small (1.8 GB) to make it possible to download for local use. However, there are much fewer parameters in the LLM and my laptop has much less compute than is available on ChatGPT. Thus, the model does not always produce the desired response

The specific prompt given to the LLM may influence the output significantly. I made sure to specify that the given text is a SQL query in my prompt. I believe this yields more consistent results, although the LLM sometimes does not give an answer